In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

np.random.seed(1)

def custom_save(function=plt.savefig, fname=""):
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print('created new directory', os.path.dirname(fname))
    try:
        # to make sure suptitle is not cut off
        function(fname, bbox_inches="tight")
    except:
        function(fname)
    print("saved as", fname)

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
CSV_DIRNAME = '../experiments/2020_09_17_white-noise-static/csv_files'
WAV_DIRNAME = '../experiments/2020_09_17_white-noise-static/export'
FS = 32000

In [ ]:
def read_csv(fname):
    def convert(row):
        arrays = ["signals_real_vect", "signals_imag_vect", "frequencies"]
        ints = ["n_mics", "n_frequencies", "timestamp"]
        for array_name in arrays:
            row[array_name] = np.fromstring(row[array_name].replace('[', '').replace(']',''), sep=' ', dtype=np.float32)
        for int_name in ints:
            row[int_name] = int(row[int_name])
            
        signals_f = row.signals_real_vect + 1j * row.signals_imag_vect
        signals_f = signals_f.reshape((row.n_mics, row.n_frequencies))
        row['signals_f'] = signals_f
        return row
    
    df = pd.read_csv(fname)
    df = df.apply(convert, axis=1)
    df.drop(["signals_real_vect", "signals_imag_vect"], axis=1, inplace=True)
    return df

def read_df(degree=0, props=True, snr=True, motors=True, source=True):
    ending = "" if degree == 0 else f"_{degree}"
    props_flag = "" if props else "no"
    motors_flag = "" if motors else "no"
    source_flag = "" if source else "no"
    snr_flag = "" if snr else "no"
    fname = f'{CSV_DIRNAME}/{motors_flag}motors_{snr_flag}snr_{props_flag}props_{source_flag}source{ending}.csv'
    print('reading', fname)
    df = read_csv(fname)
    return df

def get_spectrogram(df):
    stft = np.array([*df.loc[:, 'signals_f']]) # n_times x n_mics x n_freqs
    return np.mean(np.abs(stft), axis=1).T # average over n_mics, n_freqs x n_times
    
def add_soundlevel(df, threshold=1e-2, duration=1000):
    # detect the end time and cut fixed time before it
    spectrogram = get_spectrogram(df) # n_freqs x n_times
    sound_level = np.mean(spectrogram, axis=0) # average over n_frequencies
    df.loc[:, "sound_level"] = sound_level

    end_index = np.where(sound_level > threshold)[0][-1]
    end_time = df.timestamp[end_index]
    df.loc[:, "timestamp_s"] = (df.timestamp - end_time + duration) / 1000

def plot_spectrogram(spectrogram, label):
    plt.figure()
    plt.pcolormesh(np.log(spectrogram))
    plt.xlabel('time index [-]')
    plt.ylabel('frequency index [-]')
    plt.title(label)
    
def plot_soundlevel(df, label, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ax.semilogy(df.timestamp_s, df.sound_level, label=label)
    ax.set_xlabel('time [s]')
    ax.set_ylabel('sound level')

## Compare sound levels

In [ ]:
duration = 10 * 1e3 # miliseconds from end
degree = 0
snr = False
props = False

df_source = read_df(degree=degree, props=False, snr=snr, motors=False, source=True)
df_all = read_df(degree=degree, props=props, snr=snr, motors=True, source=True)
df_props = read_df(degree=0, props=False, snr=False, motors=True, source=False)

spectrogram_source = get_spectrogram(df_source)
spectrogram_all = get_spectrogram(df_all)
spectrogram_props = get_spectrogram(df_props)

plot_spectrogram(spectrogram_source, label="source")
plot_spectrogram(spectrogram_all, label="all")
plot_spectrogram(spectrogram_props, label="props")

add_soundlevel(df_source, duration=duration)
add_soundlevel(df_all, duration=duration)
add_soundlevel(df_props, duration=duration)

fig, ax = plt.subplots()
plot_soundlevel(df_source, "source", ax=ax)
plot_soundlevel(df_all, "all", ax=ax)
plot_soundlevel(df_props, "props", ax=ax)
plt.legend()
plt.title('audio deck signals, selected 32 frequencies')

In [ ]:
def read_df_from_wav(fname):
    import wavio
    from scipy.signal import stft
    w = wavio.read(fname)
    source_data = w.data
    
    n_buffer = 1024

    f, t, source_stft = stft(source_data, w.rate, nperseg=n_buffer, axis=0)
    source_stft = source_stft.transpose([1, 0, 2]) # channels x frequencies x times
    source_freq = np.fft.rfftfreq(n=n_buffer, d=1/w.rate)

    df = pd.DataFrame(columns=df_source.columns)
    for i in range(source_stft.shape[2]):
        df.loc[len(df), :] = {
            "index": i,
            "timestamp": i * 1024 /w.rate * 1000, # miliseconds
            "n_mics": 2,
            "topic": "measurement_mic",
            "signals_f": source_stft[:, :, i], 
            "frequencies": source_freq,
            "n_frequencies": len(source_freq)
        }
    return df

fname_source = f'{WAV_DIRNAME}/crazyflie_audio_measurements_nomotors_nosnr_noprops_source_45.wav'
fname_all =    f'{WAV_DIRNAME}/crazyflie_audio_measurements_motors_nosnr_noprops_source_45.wav'
fname_props =  f'{WAV_DIRNAME}/crazyflie_audio_measurements_motors_nosnr_noprops_nosource_45.wav'

#df_wav_all = read_df_from_wav(fname_all) 
#df_wav_source = read_df_from_wav(fname_source) 
#df_wav_props = read_df_from_wav(fname_props) 
#df_wav_all.to_pickle(fname_all.replace('.wav', '.pk'))
#df_wav_source.to_pickle(fname_source.replace('.wav', '.pk'))
#df_wav_props.to_pickle(fname_props.replace('.wav', '.pk'))

df_wav_all = pd.read_pickle(fname_all.replace('.wav', '.pk'))
df_wav_source = pd.read_pickle(fname_source.replace('.wav', '.pk'))
df_wav_props = pd.read_pickle(fname_props.replace('.wav', '.pk'))

thresh = 1e2
add_soundlevel(df_wav_source, threshold=thresh)
add_soundlevel(df_wav_all, threshold=thresh)
add_soundlevel(df_wav_props, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_source, label="source", ax=ax)
plot_soundlevel(df_wav_all, label="all", ax=ax)
plot_soundlevel(df_wav_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, all 512 frequencies')

In [ ]:
def select_bins(row):
    signals_f = row.signals_f[:, bins_wav_selected]
    frequencies = row.frequencies[bins_wav_selected]
    row.frequencies = frequencies
    row.n_frequencies = len(frequencies)
    return row

freqs_selected = df_source.loc[0, 'frequencies']
freqs_wav = df_wav_source.loc[0, 'frequencies']
bins_wav_selected = [np.argmin(np.abs(freq - freqs_wav)) for freq in freqs_selected]

df_wav_selected_all = df_wav_all.apply(select_bins, axis=1)
df_wav_selected_source = df_wav_source.apply(select_bins, axis=1)
df_wav_selected_props = df_wav_props.apply(select_bins, axis=1)

print(f"reduced from {len(df_wav_all.loc[0, 'frequencies'])} to {len(df_wav_selected_all.loc[0, 'frequencies'])}")

thresh = 1e2
add_soundlevel(df_wav_selected_source, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_all, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_props, duration=duration, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_selected_source, label="source", ax=ax)
plot_soundlevel(df_wav_selected_all, label="all", ax=ax)
plot_soundlevel(df_wav_selected_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, selected 32 frequencies')

## Beamforming

In [ ]:
degree = 20
props = False
snr = False
motors = False
source = True
gt_degrees = 180 - degree
duration = 10 * 1e3 # miliseconds

df = read_df(degree=degree, props=props, snr=snr, motors=motors, source=source)
add_soundlevel(df, duration=duration)
df = df[(df.timestamp_s > 0) & (df.timestamp_s <= (duration / 1000))]

In [ ]:
from audio_stack.spectrum_estimator import combine_rows, normalize_rows
from audio_stack.beam_former import BeamFormer

#df = df_source[(df_source.timestamp_s > 0) & (df_source.timestamp_s <= (duration / 1000))]

combine = "sum" #"product"
normalize = None #"zero_to_one"

# TODO(FD) will be read from the topic in the future
mic_d = 0.108  # distance between mics (meters)
mic_positions = mic_d / 2 * np.c_[[1, 1], [1, -1], [-1, 1], [-1, -1]].T 

beam_former = BeamFormer(mic_positions=mic_positions)

das_spectrum = None
mvdr_spectrum = None

for i, row in df.iterrows():
    signals_f = row.signals_f
    freqs = row.frequencies

    arg_idx = np.argsort(freqs)
    freqs = freqs[arg_idx]
    signals_f = signals_f[:, arg_idx]
    
    R = beam_former.get_correlation(signals_f.T)
    
    mvdr_spectrum_all = beam_former.get_mvdr_spectrum(R, freqs) # n_frequencies x n_thetas
    das_spectrum_all = beam_former.get_das_spectrum(R, freqs) # n_frequencies x n_thetas
    
    if normalize is not None:
        das_spectrum_all = normalize_rows(das_spectrum_all, method=normalize)
        mvdr_spectrum_all = normalize_rows(mvdr_spectrum_all, method=normalize)
    
    das_spectrum_column = combine_rows(das_spectrum_all, method=combine)
    if das_spectrum is None:
        das_spectrum = das_spectrum_column
    else:
        das_spectrum = np.c_[das_spectrum, das_spectrum_column]
        
    mvdr_spectrum_column = combine_rows(mvdr_spectrum_all, method=combine)
    if mvdr_spectrum is None:
        mvdr_spectrum = mvdr_spectrum_column
    else:
        mvdr_spectrum = np.c_[mvdr_spectrum, mvdr_spectrum_column]

In [ ]:
angles = np.linspace(0, 360, das_spectrum.shape[0])

plt.figure()
plt.pcolormesh(df.timestamp_s, angles, np.log(das_spectrum))
plt.axhline(gt_degrees, color='orange')
plt.xlabel('time [s]')
plt.ylabel('angle [deg]')
plt.title(f'DAS {combine}')

plt.figure()
plt.pcolormesh(df.timestamp_s, angles, np.log(mvdr_spectrum))
plt.axhline(gt_degrees, color='orange')
plt.xlabel('time [s]')
plt.ylabel('angle [deg]')
plt.title(f'MVDR {combine}')

In [ ]:
plt.figure()
plt.pcolormesh(angles, freqs, np.log(das_spectrum_all))
plt.axvline(gt_degrees, color='orange')
plt.title('DAS')
plt.xlabel('angle [deg]')
plt.ylabel('frequency [Hz]')

plt.figure()
plt.pcolormesh(angles, freqs, np.log(mvdr_spectrum_all))
plt.axvline(gt_degrees, color='orange')
plt.title('MVDR')
plt.xlabel('angle [deg]')
plt.ylabel('frequency [Hz]')